In [ ]:
import pandas as pd
from utils import load_encrypted_xlsx
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_path = '/Users/jk1/temp/cereblink/pupillometry/data_saving/exclude_nan_outcome_False/DCI_ischemia_normalised_pupillometry_df.csv'
registry_data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/sos_sah_data/post_hoc_modified_aSAH_DATA_2009_2023_24122023.xlsx'
registry_pdms_correspondence_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/dci_sah/data/pdms_data/registry_pdms_correspondence.csv'

In [ ]:
df = pd.read_csv(data_path)
registry_df = load_encrypted_xlsx(registry_data_path)
registry_pdms_correspondence_df = pd.read_csv(registry_pdms_correspondence_path)
registry_pdms_correspondence_df['Date_birth'] = pd.to_datetime(registry_pdms_correspondence_df['Date_birth'],
                                                               format='%Y-%m-%d')
registry_pdms_correspondence_df.rename(columns={'JoinedName': 'Name'}, inplace=True)
registry_df = registry_df.merge(registry_pdms_correspondence_df, on=['SOS-CENTER-YEAR-NO.', 'Name', 'Date_birth'],
                                how='left')

In [ ]:
registry_df

In [ ]:
joined_df = df.merge(registry_df[['pNr', 'Date_Ictus', 'Date_DCI_ischemia_first_image', 'Time_DCI_ischemia_first_image', 'DCI_ischemia']], on='pNr', how='left')

In [ ]:
from utils import safe_conversion_to_datetime

joined_df['full_date_dci_ischemia'] = joined_df['Date_DCI_ischemia_first_image'].astype(str) + ' ' + \
                                             joined_df['Time_DCI_ischemia_first_image'].astype(str)
# replace NaT nan with nan
joined_df['full_date_dci_ischemia'] = joined_df['full_date_dci_ischemia'].replace('NaT nan', pd.NaT)
joined_df['full_date_dci_ischemia'] = joined_df['full_date_dci_ischemia'].apply(
    safe_conversion_to_datetime)

In [ ]:
joined_df

In [ ]:
# convert to relative time
joined_df['relative_time_dci'] = (pd.to_datetime(joined_df['full_date_dci_ischemia']) - pd.to_datetime(joined_df['Date_Ictus'])).dt.total_seconds() / (60*60*24)
joined_df['relative_time_measure'] = (pd.to_datetime(joined_df['timePupil']) - pd.to_datetime(joined_df['Date_Ictus'])).dt.total_seconds() / (60*60*24)

In [ ]:
positive_joined_df = joined_df[joined_df['DCI_ischemia'] == 1]

In [ ]:
positive_joined_df

In [ ]:
positive_joined_df.pNr.unique()

In [ ]:
all_colors_palette = sns.color_palette(['#f61067', '#049b9a', '#012D98', '#a76dfe', '#FFA987', '#E2FDFF'], n_colors=6)
all_colors_palette

In [ ]:
def plot_pupillometry_over_time(temp_df, point_plot=True):
    fig, ax = plt.subplots(figsize=(15, 6))
    
    # plot CV_r_value_normalised and CV_l_value_normalised over time
    sns.lineplot(x='relative_time_measure', y='CV_r_value_normalised', data=temp_df, ax=ax, label='Right eye', color=all_colors_palette[1], alpha=0.5)
    sns.lineplot(x='relative_time_measure', y='CV_l_value_normalised', data=temp_df, ax=ax, label='Left eye', color=all_colors_palette[2], alpha=0.5)
    
    if point_plot:
        sns.scatterplot(x='relative_time_measure', y='CV_r_value_normalised', data=temp_df, ax=ax, color=all_colors_palette[1], alpha=0.7)
        sns.scatterplot(x='relative_time_measure', y='CV_l_value_normalised', data=temp_df, ax=ax, color=all_colors_palette[2], alpha=0.7)
    
    time_of_dci = temp_df.relative_time_dci.mode()[0]
    
    # add red bar at time of DCI
    plt.axvline(x=time_of_dci, color='r', linestyle='--')
    
    plt.xlabel('Time since ictus (days)')
    plt.ylabel('Normalised CV')
    
    # xlim to 0-15
    plt.xlim(0, time_of_dci + 0.1)
    plt.show()
    
    return fig

In [ ]:
point_plot = True
temp_df = positive_joined_df[positive_joined_df.pNr == 81512]
plot_pupillometry_over_time(temp_df, point_plot)

In [ ]:
for pNr in positive_joined_df.pNr.unique():
    temp_df = positive_joined_df[positive_joined_df.pNr == pNr]
    plot_pupillometry_over_time(temp_df, point_plot)